In [1]:
import numpy as np
from keras.datasets import mnist
def load_mnist(flatten=True, normalize=True, one_hot=True, num_classes=10):
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    if flatten:
        x_train = x_train.reshape(x_train.shape[0], -1)
        x_test = x_test.reshape(x_test.shape[0], -1)
    if normalize:
        x_train = x_train / 255.0
        x_test = x_test / 255.0
    if one_hot:
        y_train = np.eye(num_classes)[y_train]
        y_test = np.eye(num_classes)[y_test]
    return x_train, y_train, x_test, y_test

In [2]:
import numpy as np
def relu(z):
    return np.maximum(0, z)
def relu_der(z):
    return (z > 0).astype(float)
def soft(z):
    exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))
    return exp_z / np.sum(exp_z, axis=1, keepdims=True)

In [3]:
def ini_para(layer_sizes):
    np.random.seed(42)
    params = {}
    L = len(layer_sizes)
    for l in range(1, L):
        params['W'+str(l)] = np.random.randn(layer_sizes[l-1], layer_sizes[l]) * 0.01
        params['b'+str(l)] = np.zeros((1, layer_sizes[l]))
    return params

In [4]:
def forward(X, params):
    cache = {'A0': X}
    L = len(params)//2
    for l in range(1, L):
        Z = np.dot(cache['A'+str(l-1)], params['W'+str(l)]) + params['b'+str(l)]
        A = relu(Z)
        cache['Z'+str(l)] = Z
        cache['A'+str(l)] = A
    ZL = np.dot(cache['A'+str(L-1)], params['W'+str(L)]) + params['b'+str(L)]
    AL = soft(ZL)
    cache['Z'+str(L)] = ZL
    cache['A'+str(L)] = AL
    return AL, cache

In [5]:
def comp_loss(Y_pred, Y_true):
    m = Y_true.shape[0]
    loss = -np.sum(Y_true * np.log(Y_pred + 1e-8)) / m
    return loss

In [6]:
def backward(Y_pred, Y_true, params, cache, learning_rate=0.01):
    grads = {}
    L = len(params)//2
    m = Y_true.shape[0]
    dZ = Y_pred - Y_true
    grads['dW'+str(L)] = np.dot(cache['A'+str(L-1)].T, dZ)/m
    grads['db'+str(L)] = np.sum(dZ, axis=0, keepdims=True)/m
    dA_prev = np.dot(dZ, params['W'+str(L)].T)
    for l in reversed(range(1, L)):
        dZ = dA_prev * relu_der(cache['Z'+str(l)])
        grads['dW'+str(l)] = np.dot(cache['A'+str(l-1)].T, dZ)/m
        grads['db'+str(l)] = np.sum(dZ, axis=0, keepdims=True)/m
        if l > 1:
            dA_prev = np.dot(dZ, params['W'+str(l)].T)
    for l in range(1, L+1):
        params['W'+str(l)] -= learning_rate * grads['dW'+str(l)]
        params['b'+str(l)] -= learning_rate * grads['db'+str(l)]

In [7]:
def train(X_train, Y_train, X_test, Y_test, hidden_layers=[128,64], lr=0.01, epochs=20):
    layer_sizes = [X_train.shape[1]] + hidden_layers + [10]
    params = ini_para(layer_sizes)
    for epoch in range(epochs):
        Y_pred, cache = forward(X_train, params)
        loss = comp_loss(Y_pred, Y_train)
        backward(Y_pred, Y_train, params, cache, learning_rate=lr)
        if (epoch+1) % 5 == 0:
            print(f"Epoch {epoch+1}/{epochs}, Loss: {loss:.4f}")
    Y_pred_test, _ = forward(X_test, params)
    accuracy = np.mean(np.argmax(Y_pred_test, axis=1) == np.argmax(Y_test, axis=1))
    print(f"Test Accuracy: {accuracy*100:.2f}%")
    return params

In [8]:
import numpy as np
import wandb
wandb.init(
    project="mnist-q8_a",
    name="CE_vs_Squared_Error"
)
from keras.datasets import mnist
def load_mnist(flatten=True, normalize=True, one_hot=True, num_classes=10):
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    if flatten:
        x_train = x_train.reshape(x_train.shape[0], -1)
        x_test = x_test.reshape(x_test.shape[0], -1)
    if normalize:
        x_train = x_train / 255.0
        x_test = x_test / 255.0
    if one_hot:
        y_train = np.eye(num_classes)[y_train]
        y_test = np.eye(num_classes)[y_test]
    return x_train, y_train, x_test, y_test
class Activate:
    @staticmethod
    def relu(Z):
        return np.maximum(0, Z)
    @staticmethod
    def relu_der(Z):
        return (Z > 0).astype(float)
    @staticmethod
    def softmax(Z):
        expZ = np.exp(Z - np.max(Z, axis=1, keepdims=True))
        return expZ / np.sum(expZ, axis=1, keepdims=True)
class FNN:
    def __init__(self, input_size, hidden_layers, output_size, seed=42):
        np.random.seed(seed)
        self.layers = [input_size] + hidden_layers + [output_size]
        self.L = len(self.layers) - 1
        self.weights = {}
        self.biases = {}
        for l in range(1, self.L+1):
            limit = np.sqrt(6 / (self.layers[l-1] + self.layers[l]))
            self.weights['W'+str(l)] = np.random.uniform(-limit, limit, (self.layers[l-1], self.layers[l]))
            self.biases['b'+str(l)] = np.zeros((1, self.layers[l]))
        self.opt_caches = {}
    def forward(self, X):
        self.cache = {'A0': X}
        for l in range(1, self.L):
            Z = np.dot(self.cache['A'+str(l-1)], self.weights['W'+str(l)]) + self.biases['b'+str(l)]
            A = Activate.relu(Z)
            self.cache['Z'+str(l)] = Z
            self.cache['A'+str(l)] = A
        ZL = np.dot(self.cache['A'+str(self.L-1)], self.weights['W'+str(self.L)]) + self.biases['b'+str(self.L)]
        AL = Activate.softmax(ZL)
        self.cache['Z'+str(self.L)] = ZL
        self.cache['A'+str(self.L)] = AL
        return AL
    def backward(self, Y_true, loss_type="cross_entropy"):
        m = Y_true.shape[0]
        self.grads = {}
        Y_pred = self.cache['A'+str(self.L)]
        if loss_type == "cross_entropy":
          dZ = Y_pred - Y_true
        elif loss_type == "squared_error":
          dZ = (Y_pred - Y_true) * Y_pred * (1 - Y_pred)

        self.grads['dW'+str(self.L)] = np.dot(self.cache['A'+str(self.L-1)].T, dZ)/m
        self.grads['db'+str(self.L)] = np.sum(dZ, axis=0, keepdims=True)/m
        dA_prev = np.dot(dZ, self.weights['W'+str(self.L)].T)
        for l in reversed(range(1, self.L)):
            dZ = dA_prev * Activate.relu_der(self.cache['Z'+str(l)])
            self.grads['dW'+str(l)] = np.dot(self.cache['A'+str(l-1)].T, dZ)/m
            self.grads['db'+str(l)] = np.sum(dZ, axis=0, keepdims=True)/m
            if l > 1:
                dA_prev = np.dot(dZ, self.weights['W'+str(l)].T)
    def update_parameters(self, lr=0.01, optimizer='sgd', beta1=0.9, beta2=0.999, epsilon=1e-8, t=1):
        for l in range(1, self.L+1):
            dW = self.grads['dW'+str(l)]
            db = self.grads['db'+str(l)]
            if optimizer == 'sgd':
                self.weights['W'+str(l)] -= lr * dW
                self.biases['b'+str(l)] -= lr * db
            elif optimizer == 'momentum':
                if 'vW'+str(l) not in self.opt_caches:
                    self.opt_caches['vW'+str(l)] = np.zeros_like(dW)
                    self.opt_caches['vb'+str(l)] = np.zeros_like(db)
                self.opt_caches['vW'+str(l)] = beta1*self.opt_caches['vW'+str(l)] + (1-beta1)*dW
                self.opt_caches['vb'+str(l)] = beta1*self.opt_caches['vb'+str(l)] + (1-beta1)*db
                self.weights['W'+str(l)] -= lr * self.opt_caches['vW'+str(l)]
                self.biases['b'+str(l)] -= lr * self.opt_caches['vb'+str(l)]
            elif optimizer == 'nesterov':
                if 'vW'+str(l) not in self.opt_caches:
                    self.opt_caches['vW'+str(l)] = np.zeros_like(dW)
                    self.opt_caches['vb'+str(l)] = np.zeros_like(db)
                vW_prev = self.opt_caches['vW'+str(l)].copy()
                vb_prev = self.opt_caches['vb'+str(l)].copy()
                self.opt_caches['vW'+str(l)] = beta1*self.opt_caches['vW'+str(l)] + lr*dW
                self.opt_caches['vb'+str(l)] = beta1*self.opt_caches['vb'+str(l)] + lr*db
                self.weights['W'+str(l)] -= beta1*vW_prev + (1+beta1)*(self.opt_caches['vW'+str(l)] - vW_prev)
                self.biases['b'+str(l)] -= beta1*vb_prev + (1+beta1)*(self.opt_caches['vb'+str(l)] - vb_prev)
            elif optimizer == 'rmsprop':
                if 'sW'+str(l) not in self.opt_caches:
                    self.opt_caches['sW'+str(l)] = np.zeros_like(dW)
                    self.opt_caches['sb'+str(l)] = np.zeros_like(db)
                self.opt_caches['sW'+str(l)] = beta2*self.opt_caches['sW'+str(l)] + (1-beta2)*(dW**2)
                self.opt_caches['sb'+str(l)] = beta2*self.opt_caches['sb'+str(l)] + (1-beta2)*(db**2)
                self.weights['W'+str(l)] -= lr * dW / (np.sqrt(self.opt_caches['sW'+str(l)]) + epsilon)
                self.biases['b'+str(l)] -= lr * db / (np.sqrt(self.opt_caches['sb'+str(l)]) + epsilon)
            elif optimizer == 'adam':
                if 'vW'+str(l) not in self.opt_caches:
                    self.opt_caches['vW'+str(l)] = np.zeros_like(dW)
                    self.opt_caches['vb'+str(l)] = np.zeros_like(db)
                    self.opt_caches['sW'+str(l)] = np.zeros_like(dW)
                    self.opt_caches['sb'+str(l)] = np.zeros_like(db)
                self.opt_caches['vW'+str(l)] = beta1*self.opt_caches['vW'+str(l)] + (1-beta1)*dW
                self.opt_caches['vb'+str(l)] = beta1*self.opt_caches['vb'+str(l)] + (1-beta1)*db
                self.opt_caches['sW'+str(l)] = beta2*self.opt_caches['sW'+str(l)] + (1-beta2)*(dW**2)
                self.opt_caches['sb'+str(l)] = beta2*self.opt_caches['sb'+str(l)] + (1-beta2)*(db**2)
                vW_corr = self.opt_caches['vW'+str(l)] / (1 - beta1**t)
                vb_corr = self.opt_caches['vb'+str(l)] / (1 - beta1**t)
                sW_corr = self.opt_caches['sW'+str(l)] / (1 - beta2**t)
                sb_corr = self.opt_caches['sb'+str(l)] / (1 - beta2**t)
                self.weights['W'+str(l)] -= lr * vW_corr / (np.sqrt(sW_corr) + epsilon)
                self.biases['b'+str(l)] -= lr * vb_corr / (np.sqrt(sb_corr) + epsilon)
            elif optimizer == 'nadam':
                if 'vW'+str(l) not in self.opt_caches:
                    self.opt_caches['vW'+str(l)] = np.zeros_like(dW)
                    self.opt_caches['vb'+str(l)] = np.zeros_like(db)
                    self.opt_caches['sW'+str(l)] = np.zeros_like(dW)
                    self.opt_caches['sb'+str(l)] = np.zeros_like(db)
                vW_prev = self.opt_caches['vW'+str(l)].copy()
                vb_prev = self.opt_caches['vb'+str(l)].copy()
                self.opt_caches['vW'+str(l)] = beta1*self.opt_caches['vW'+str(l)] + (1-beta1)*dW
                self.opt_caches['vb'+str(l)] = beta1*self.opt_caches['vb'+str(l)] + (1-beta1)*db
                self.opt_caches['sW'+str(l)] = beta2*self.opt_caches['sW'+str(l)] + (1-beta2)*(dW**2)
                self.opt_caches['sb'+str(l)] = beta2*self.opt_caches['sb'+str(l)] + (1-beta2)*(db**2)
                vW_corr = (beta1*vW_prev + (1+beta1)*self.opt_caches['vW'+str(l)]/(1-beta1**t)) / (1 - beta1**t)
                vb_corr = (beta1*vb_prev + (1+beta1)*self.opt_caches['vb'+str(l)]/(1-beta1**t)) / (1 - beta1**t)
                sW_corr = self.opt_caches['sW'+str(l)] / (1 - beta2**t)
                sb_corr = self.opt_caches['sb'+str(l)] / (1 - beta2**t)
                self.weights['W'+str(l)] -= lr * vW_corr / (np.sqrt(sW_corr) + epsilon)
                self.biases['b'+str(l)] -= lr * vb_corr / (np.sqrt(sb_corr) + epsilon)
    def cross_entropy_loss(self, Y_pred, Y_true):
        m = Y_true.shape[0]
        return -np.sum(Y_true*np.log(Y_pred + 1e-8))/m
    def squared_error_loss(self, Y_pred, Y_true):
        return np.mean(np.sum((Y_pred - Y_true) ** 2, axis=1))
    def compute_loss(self, Y_pred, Y_true):
        return self.cross_entropy_loss(Y_pred, Y_true)
    def train(self, X_train, Y_train, X_test, Y_test, epochs=10, batch_size=64, lr=0.01, optimizer='sgd'):
        num_samples = X_train.shape[0]
        loss_history_ce = []
        loss_history_se = []
        for epoch in range(epochs):
          perm = np.random.permutation(num_samples)
          X_shuf, Y_shuf = X_train[perm], Y_train[perm]
          epoch_ce, epoch_se, num_batches = 0, 0, 0
          for i in range(0, num_samples, batch_size):
            Xb = X_shuf[i:i+batch_size]
            Yb = Y_shuf[i:i+batch_size]
            Y_pred = self.forward(Xb)
            loss_ce = self.cross_entropy_loss(Y_pred, Yb)
            loss_se = self.squared_error_loss(Y_pred, Yb)
            self.backward(Yb, loss_type="cross_entropy")
            self.update_parameters(lr=lr, optimizer=optimizer, t=epoch+1)
            epoch_ce += loss_ce
            epoch_se += loss_se
            num_batches += 1
          epoch_ce /= num_batches
          epoch_se /= num_batches
          loss_history_ce.append(epoch_ce)
          loss_history_se.append(epoch_se)
          Y_test_pred = self.forward(X_test)
          test_acc = np.mean(
          np.argmax(Y_test_pred, axis=1) == np.argmax(Y_test, axis=1)
          )
          wandb.log({
              "epoch": epoch,
              "cross_entropy_loss": epoch_ce,
              "squared_error_loss": epoch_se,
              "test_accuracy": test_acc
          })
        wandb.log({
                "loss_comparison": wandb.plot.line_series(
                xs=list(range(len(loss_history_ce))),
                ys=[loss_history_ce, loss_history_se],
                keys=["Cross Entropy", "Squared Error"],
                title="Loss Comparison: Cross-Entropy vs Squared Error",
                xname="Epoch"
                )
                })

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:wandb: WARNING Invalid choice
wandb: Enter your choice:

 2


wandb: You chose 'Use an existing W&B account'
wandb: Logging into https://api.wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: Find your API key here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 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


wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: manees-6732 (vaishalinir-ymc2022-chennai-institute-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [10]:
X_train, Y_train, X_test, Y_test = load_mnist()
configs = [
    {
        "name": "Config_1_Best_Overall",
        "hidden_layers": [128, 128, 128],
        "optimizer": "adam",
        "lr": 0.001,
        "batch_size": 64
    },
    {
        "name": "Config_2_Deep_Feature",
        "hidden_layers": [128, 128, 128, 128, 128],
        "optimizer": "nadam",
        "lr": 0.001,
        "batch_size": 64
    },
    {
        "name": "Config_3_Lightweight",
        "hidden_layers": [64, 64],
        "optimizer": "adam",
        "lr": 0.001,
        "batch_size": 32
    }
]
for cfg in configs:
    wandb.init(
        project="mnist-q10",
        name=cfg["name"],
        config=cfg
    )
    model = FNN(
        input_size=784,
        hidden_layers=cfg["hidden_layers"],
        output_size=10
    )
    model.train(
        X_train, Y_train,
        X_test, Y_test,
        epochs=10,
        batch_size=cfg["batch_size"],
        lr=cfg["lr"],
        optimizer=cfg["optimizer"]
    )
    wandb.finish()

cross_entropy_loss,█▄▃▂▂▂▁▁▁▁
epoch,▁▂▃▃▄▅▆▆▇█
squared_error_loss,█▄▃▂▂▂▁▁▁▁
test_accuracy,▁▅▇▇▇▆████
cross_entropy_loss,0.02489
epoch,9
squared_error_loss,0.01125
test_accuracy,0.9777


cross_entropy_loss,█▃▂▂▂▂▁▁▁▁
epoch,▁▂▃▃▄▅▆▆▇█
squared_error_loss,█▃▂▂▂▂▁▁▁▁
test_accuracy,▁▅▆▇▇▇████
cross_entropy_loss,0.11875
epoch,9
squared_error_loss,0.0544
test_accuracy,0.9513


cross_entropy_loss,█▄▃▃▂▂▂▁▁▁
epoch,▁▂▃▃▄▅▆▆▇█
squared_error_loss,█▄▃▃▂▂▂▁▁▁
test_accuracy,▁▄▅▆▆▆▇▇██
cross_entropy_loss,0.06365
epoch,9
squared_error_loss,0.02912
test_accuracy,0.9731
